In [1]:
import numpy as np
import modin.pandas as mipd
import os
import time
import pandas as pd
from tqdm import tqdm
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import numpy as np
import gc
import seaborn as sns

from matplotlib import pyplot as plt
from scipy.signal import savgol_filter

In [2]:
### Faster
time_resolution = 21
time_window = 5

c_path = '../network_data_smoothed'+str(time_resolution+time_window)+'/commits/'
e_path = '../network_data_smoothed'+str(time_resolution+time_window)+'/emails/'
#c_path = './network_data/commits/'

projects = os.listdir(c_path)
project_names = [x.split('__')[0] for x in projects]
project_freqs = pd.Series(project_names)

project_names = pd.Series(project_names).drop_duplicates().values
project_freqs.describe()

count     6014
unique     259
top       wave
freq        81
dtype: object

In [3]:
df_tech_overlaps = pd.read_csv("../result_csv/tech_overlaps_smoothed"+str(time_resolution+time_window)+".csv")
df_social_overlaps = pd.read_csv("../result_csv/social_overlaps_smoothed"+str(time_resolution+time_window)+".csv")
df_tech_edgesize = pd.read_csv("../result_csv/tech_edgesize_smoothed"+str(time_resolution+time_window)+".csv")
df_social_edgesize = pd.read_csv("../result_csv/social_edgesize_smoothed"+str(time_resolution+time_window)+".csv")

all_graduated = np.load('../all_graduated.npy').tolist()
#all_graduated = [x.lower() for x in all_graduated]
all_retired = np.load('../all_retired.npy').tolist()
#all_retired = [x.lower() for x in all_retired]

all_graduated = [x.lower() for x in all_graduated if x.lower() in df_tech_overlaps.columns and x.lower() in df_social_overlaps.columns]
all_retired = [x.lower() for x in all_retired if x.lower() in df_tech_overlaps.columns and x.lower() in df_social_overlaps.columns]

In [5]:
df_social_edgesize.T

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
openwebbeans,10,22,24,28,24,24,16,18,10,6,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
jena,128,82,114,96,100,86,132,110,80,80,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
etch,14,16,28,30,50,52,2,34,18,24,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
wave,446,414,286,320,268,302,238,78,120,102,...,22,16,8,10,4,2,4,0,28,24
shindig,44,258,334,306,428,452,458,476,428,264,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
muse,18,0,8,0,2,2,8,12,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
avalon,192,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
onami,68,34,70,66,88,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
nutch,0,92,396,272,186,188,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [13]:
df_social_edgesize.T.sort_values(by=[x for x in range(99)], ascending=False).head(30)

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
flex,5178,3122,1546,1326,662,662,848,804,926,644,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
harmony,2130,398,324,98,130,208,274,204,166,202,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
openoffice,1562,1694,1340,1688,1172,1530,1420,1474,1612,992,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
struts,1494,1042,1182,1664,1424,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
myfaces,1176,1136,1110,1300,1346,1486,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
stratos,1172,366,124,162,230,202,270,220,230,212,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
geronimo,1170,340,358,214,420,234,124,192,92,104,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
geode,666,482,346,262,224,240,232,296,390,460,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
drill,574,280,166,70,54,42,84,132,70,52,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
cloudstack,528,748,964,844,1336,1086,1220,1458,1146,1714,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [14]:
df_tech_edgesize.T.sort_values(by=[x for x in range(99)], ascending=False).head(30)

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
jclouds,8330,264,1003,524,816,984,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
ignite,7680,28888,59840,27903,26824,10786,11713,11000,16589,4534,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
corinthia,5389,14,732,2106,1314,71,101,41,4965,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
druid,4399,651,3899,1196,1057,480,169,632,491,700,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
griffin,3737,58,1155,390,1270,250,224,14,404,602,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
geode,3584,32,6379,3422,1699,1174,15351,26076,14595,6288,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
hawq,3513,149,1133,820,813,447,210,433,971,1861,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
systemml,2884,207,330,156,112,485,142,159,257,204,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
tinkerpop,2807,1069,986,1018,1414,731,342,395,736,208,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
stratos,2781,555,359,189,770,772,2200,1444,1182,1830,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
